# Simple Voice Bot with FAISS RAG


In [ ]:
# Install dependencies (run once)
# !pip install faiss-cpu sentence-transformers openai PyPDF2 sounddevice soundfile scipy

In [ ]:
import os
import faiss
import numpy as np
import PyPDF2
from sentence_transformers import SentenceTransformer
from openai import OpenAI

# Your API key
api_key = "<plese enter your openai key>"  # Replace with your key
client = OpenAI(api_key=api_key)

# Initialize embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

/Users/amit.yadav/Documents/docs/BIA/AgenticAIAug2025/Speech and avatar/voice_bot_code/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# ========== PDF Processing & FAISS Index ==========

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() + "\n"
            
    return text

def chunk_text(text, chunk_size=1000, overlap=200):
    chunks = []
    start = 0
    while start < len(text): ###4000
        end = start + chunk_size ###0+1000 =1000+800=1800
        chunks.append(text[start:end])
        start = end - overlap###1800-200 = 1600
    return [c for c in chunks if len(c.strip()) > 50]

def build_faiss_index(pdf_directory):
    """Build FAISS index from PDFs in directory"""
    all_chunks = []
    metadata = []
    
    for filename in os.listdir(pdf_directory):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(pdf_directory, filename)
            print(f"Processing: {filename}")
            text = extract_text_from_pdf(pdf_path)
            chunks = chunk_text(text)
            all_chunks.extend(chunks)
            metadata.extend([{'source': filename, 'chunk_id': i} for i in range(len(chunks))])
    
    # Create embeddings
    print(f"Creating embeddings for {len(all_chunks)} chunks...")
    embeddings = embedding_model.encode(all_chunks, show_progress_bar=True)
    embeddings = np.array(embeddings).astype('float32')
    
    # Build FAISS index
    dimension = embeddings.shape[1]  # 384 for all-MiniLM-L6-v2
    index = faiss.IndexFlatIP(dimension)  # Inner product (cosine similarity after normalization)
    faiss.normalize_L2(embeddings)  # Normalize for cosine similarity
    index.add(embeddings)
    
    print(f"FAISS index built with {index.ntotal} vectors")
    return index, all_chunks, metadata

# Build the index
pdf_directory = "/Users/amit.yadav/Documents/docs/BIA/AgenticAIAug2025/Speech and avatar/voice_bot_code/voicebot/data"
faiss_index, chunks, chunk_metadata = build_faiss_index(pdf_directory)

Processing: statistics.pdf
Creating embeddings for 36 chunks...


Batches: 100%|██████████| 2/2 [00:00<00:00,  3.33it/s]

FAISS index built with 36 vectors


In [7]:
# ========== RAG Retrieval ==========

def retrieve_documents(query, top_k=3):
    """Retrieve relevant chunks using FAISS"""
    query_embedding = embedding_model.encode([query])
    query_embedding = np.array(query_embedding).astype('float32')
    faiss.normalize_L2(query_embedding)
    
    scores, indices = faiss_index.search(query_embedding, top_k)
    
    results = []
    for score, idx in zip(scores[0], indices[0]):
        if idx != -1:  # Valid result
            results.append({
                'text': chunks[idx],
                'source': chunk_metadata[idx]['source'],
                'score': float(score)
            })
    return results

# Test retrieval
test_results = retrieve_documents("What is Bayesian regression?")
print(f"Found {len(test_results)} relevant chunks")
for r in test_results:
    print(f"  - Score: {r['score']:.3f}, Source: {r['source']}")

Found 3 relevant chunks
  - Score: 0.317, Source: statistics.pdf
  - Score: 0.298, Source: statistics.pdf
  - Score: 0.286, Source: statistics.pdf


In [8]:
# ========== RAG + GPT-4o (Text) ==========

def ask_with_rag(query):
    """Answer query using RAG + GPT-4o"""
    docs = retrieve_documents(query, top_k=3)
    
    context = "\n\n".join([f"[{d['source']}]: {d['text'][:800]}" for d in docs])
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful AI instructor. Use the provided context to answer questions clearly."},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"}
        ],
        max_tokens=500
    )
    return response.choices[0].message.content

# Test
answer = ask_with_rag("Explain Bayesian regression simply")
print(answer)

Bayesian regression is a statistical method that extends traditional regression analysis by incorporating prior beliefs or information into the model. Here's a simple explanation:

1. **Basic Idea**: Unlike traditional (frequentist) regression, which estimates a single "best" set of parameters based solely on the observed data, Bayesian regression combines information from the data with prior beliefs to form a posterior distribution of the parameters.

2. **Prior Information**: Before observing any data, you have some beliefs or assumptions about the parameters in the form of a "prior distribution." This could be based on previous studies, expert knowledge, or other relevant information.

3. **Data and Likelihood**: You gather data and determine how likely this data is given different parameter values. This is called the likelihood function.

4. **Bayes' Theorem**: Bayesian regression uses Bayes' Theorem to update your prior beliefs with the observed data, resulting in a "posterior dis

In [9]:
# ========== Simple Voice Bot (Non-blocking) ==========

import sounddevice as sd
import soundfile as sf
import tempfile
from pathlib import Path

SAMPLE_RATE = 16000

def record_audio(duration=5):
    """Record audio for specified duration"""
    print(f"🎤 Recording for {duration} seconds... Speak now!")
    audio = sd.rec(int(duration * SAMPLE_RATE), samplerate=SAMPLE_RATE, channels=1, dtype='int16')
    sd.wait()  # Wait until recording is finished
    print("✅ Recording complete")
    return audio

def transcribe_audio(audio_data):
    """Transcribe audio using Whisper API"""
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as f:
        sf.write(f.name, audio_data, SAMPLE_RATE)
        with open(f.name, "rb") as audio_file:
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file
            )
        Path(f.name).unlink()  # Clean up temp file
    return transcript.text

def text_to_speech(text):
    """Convert text to speech and play it"""
    response = client.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=text
    )
    
    with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as f:
        f.write(response.content)
        # Play the audio
        data, sr = sf.read(f.name)
        sd.play(data, sr)
        sd.wait()
        Path(f.name).unlink()

def voice_chat(duration=5):
    """Complete voice interaction: listen -> RAG -> respond"""
    # 1. Record user's question
    audio = record_audio(duration)
    
    # 2. Transcribe
    print("🔄 Transcribing...")
    user_text = transcribe_audio(audio)
    print(f"📝 You said: {user_text}")
    
    # 3. Get RAG-enhanced answer
    print("🤔 Thinking...")
    answer = ask_with_rag(user_text)
    print(f"🤖 Answer: {answer}")
    
    # 4. Speak the answer
    print("🔊 Speaking...")
    text_to_speech(answer)
    print("✅ Done!")
    
    return user_text, answer

In [12]:
# ========== Run Voice Bot ==========
# Run this cell and speak your question

user_question, bot_answer = voice_chat(duration=5)  # 5 seconds to speak

🎤 Recording for 5 seconds... Speak now!
✅ Recording complete
🔄 Transcribing...
📝 You said: Tell me about Bayesian statistics.
🤔 Thinking...
🤖 Answer: The provided context does not specifically cover Bayesian statistics, but I can provide a general overview.

Bayesian statistics is a branch of statistics that uses Bayes' theorem to update the probability for a hypothesis as more evidence or information becomes available. It contrasts with frequentist statistics, which typically relies on fixed models and frequencies for inference.

Key aspects of Bayesian statistics include:

1. **Prior Probability**: Before considering any new data, Bayesian statistics starts with a prior probability or distribution, which represents the initial belief about the parameters.

2. **Likelihood**: The likelihood function represents the probability of observing the data given a set of parameters.

3. **Posterior Probability**: After observing the data, the posterior probability distribution is calculated us

KeyboardInterrupt: 

In [11]:
# ========== Interactive Loop (Optional) ==========
# For continuous conversation, run this:

def interactive_voice_bot():
    """Run voice bot in a loop - type 'quit' to exit"""
    print("🎙️ Voice Bot Ready!")
    print("Press Enter to speak, or type 'quit' to exit")
    
    while True:
        user_input = input("\n[Press Enter to speak, or type 'quit']: ")
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
        voice_chat(duration=5)

# Uncomment to run:
# interactive_voice_bot()